<a href="https://colab.research.google.com/github/praveenbenedict/tf-developer-certificate-coursera/blob/master/3%20-%20Natural%20Language%20Processing%20in%20Tensorflow/Week%202/2.1.1%20-%20IMDB_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
import tensorflow_datasets as tfds

In [3]:
imdb, imdb_info = tfds.load('imdb_reviews', with_info = True, as_supervised = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete26O77J/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete26O77J/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete26O77J/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
import numpy as np

In [5]:
train_data, test_data = imdb['train'], imdb['test']

In [9]:
for item in train_data:
  print(item[0])
  print(item[1])
  break

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)


In [13]:
training_sentences, training_labels, testing_sentences, testing_labels = [], [], [], []

In [14]:
for s, l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())

In [15]:
for s, l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

In [17]:
print(training_sentences[0])
print(type(training_sentences[0]))

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
<class 'str'>


In [18]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [25]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)

In [26]:
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

In [27]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [44]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [45]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=16, input_length=max_length),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(6, activation = 'relu'),
  tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [46]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 120, 16)           160000    
                                                                 
 flatten_3 (Flatten)         (None, 1920)              0         
                                                                 
 dense_6 (Dense)             (None, 6)                 11526     
                                                                 
 dense_7 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [52]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=16, input_length=max_length),
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(6, activation = 'relu'),
  tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [53]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 120, 16)           160000    
                                                                 
 global_average_pooling1d_2   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_10 (Dense)            (None, 6)                 102       
                                                                 
 dense_11 (Dense)            (None, 1)                 7         
                                                                 
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [54]:
model.compile(
    optimizer = 'rmsprop',
    loss = 'binary_crossentropy',
    metrics = 'accuracy'
)

In [55]:
model.fit(
    padded, training_labels_final, batch_size = 16, epochs = 10, verbose = 2, validation_data = (testing_padded, testing_labels_final)
)

Epoch 1/10
1563/1563 - 9s - loss: 0.6504 - accuracy: 0.6344 - val_loss: 0.5339 - val_accuracy: 0.7792 - 9s/epoch - 6ms/step
Epoch 2/10
1563/1563 - 9s - loss: 0.4427 - accuracy: 0.8062 - val_loss: 0.3711 - val_accuracy: 0.8394 - 9s/epoch - 6ms/step
Epoch 3/10
1563/1563 - 7s - loss: 0.3518 - accuracy: 0.8487 - val_loss: 0.3468 - val_accuracy: 0.8495 - 7s/epoch - 4ms/step
Epoch 4/10
1563/1563 - 8s - loss: 0.3162 - accuracy: 0.8668 - val_loss: 0.3321 - val_accuracy: 0.8565 - 8s/epoch - 5ms/step
Epoch 5/10
1563/1563 - 7s - loss: 0.2965 - accuracy: 0.8776 - val_loss: 0.3301 - val_accuracy: 0.8575 - 7s/epoch - 5ms/step
Epoch 6/10
1563/1563 - 7s - loss: 0.2829 - accuracy: 0.8832 - val_loss: 0.3333 - val_accuracy: 0.8549 - 7s/epoch - 5ms/step
Epoch 7/10
1563/1563 - 8s - loss: 0.2732 - accuracy: 0.8910 - val_loss: 0.3347 - val_accuracy: 0.8552 - 8s/epoch - 5ms/step
Epoch 8/10
1563/1563 - 7s - loss: 0.2650 - accuracy: 0.8936 - val_loss: 0.3390 - val_accuracy: 0.8559 - 7s/epoch - 4ms/step
Epoch 9/

In [42]:
padded[0]

array([   0,    0,    0,   12,   14,   33,  425,  392,   18,   90,   28,
          1,    9,   32, 1366, 3585,   40,  486,    1,  197,   24,   85,
        154,   19,   12,  213,  329,   28,   66,  247,  215,    9,  477,
         58,   66,   85,  114,   98,   22, 5675,   12, 1322,  643,  767,
         12,   18,    7,   33,  400, 8170,  176, 2455,  416,    2,   89,
       1231,  137,   69,  146,   52,    2,    1, 7577,   69,  229,   66,
       2933,   16,    1, 2904,    1,    1, 1479, 4940,    3,   39, 3900,
        117, 1584,   17, 3585,   14,  162,   19,    4, 1231,  917, 7917,
          9,    4,   18,   13,   14, 4139,    5,   99,  145, 1214,   11,
        242,  683,   13,   48,   24,  100,   38,   12, 7181, 5515,   38,
       1366,    1,   50,  401,   11,   98, 1197,  867,  141,   10],
      dtype=int32)

In [43]:
training_labels[0]

0